In [12]:
import os
import random
import torch
from d2l import torch as d2l

# 用于预训练bert的数据集

1. 先读取WikiText文本、分句，得到段落列表。
2. 对每对句子，采样“下一句预测”正负例，并加BERT的特殊符号。
3. 随机mask部分token，构造MLM样本。
4. 所有输入统一pad到定长。
5. 打包成PyTorch Dataset，支持dataloader批量加载。
6. 最终返回能直接用于BERT预训练的数据流和词表。

## 1. 下载并读取 WikiText-2 语料

In [13]:
# 注册WikiText-2数据集的下载信息（URL与校验码）
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')

def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')  # 拼接出训练集路径
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # 每行按'.'切分成句子片段，再统一转成小写
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)  # 打乱段落顺序，增强训练多样性
    return paragraphs

## 2 生成下一句预测任务的数据
### Step 1：采样一对句子，决定它们是不是前后连续

In [14]:
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:         # 50%的概率，保持原有句子顺序（正例）
        is_next = True
    else:                             # 另外50%的概率，随机选一条不是下一个的句子（负例）
        # paragraphs是嵌套的三层列表，random.choice嵌套随机抽一个句子
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next


### Step 2：用上面的采样逻辑，把一个段落里的每一对相邻句子都处理一下

In [15]:
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # 这里tokens_a, tokens_b都是字符串（后续还会被切分成词）
        # 句子加上BERT特殊符号后不能超过最大长度max_len，否则跳过
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        # d2l.get_tokens_and_segments会加<cls>、<sep>并返回片段编码
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph



## 3. 生成遮蔽语言模型（MLM）任务的数据
### Step 1：根据BERT策略随机选择并替换词

In [16]:
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab):
    mlm_input_tokens = [token for token in tokens]     # 复制一份token
    pred_positions_and_labels = []
    random.shuffle(candidate_pred_positions)           # 打乱可mask的位置
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        if random.random() < 0.8:                      # 80%概率替换成<mask>
            masked_token = '<mask>'
        else:
            if random.random() < 0.5:                  # 10%保持原token
                masked_token = tokens[mlm_pred_position]
            else:                                      # 10%替换成随机词
                masked_token = random.choice(vocab.idx_to_token)
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels


### Step 2：对输入token序列生成MLM任务训练所需信息

In [17]:
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    for i, token in enumerate(tokens):
        if token in ['<cls>', '<sep>']:        # 特殊符号不能被mask
            continue
        candidate_pred_positions.append(i)
    num_mlm_preds = max(1, round(len(tokens) * 0.15))   # 取15%的token用于mask
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels, key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]    # 被mask的位置
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]   # 被mask的原词
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]


## 4. 文本转为BERT预训练数据的完整流程
### Step 3：所有任务输出补齐为统一长度（padding）

In [18]:
def _pad_bert_inputs(examples, max_len, vocab):
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    for (token_ids, pred_positions, mlm_pred_label_ids, segments, is_next) in examples:
        # tokens与segments都要补齐max_len
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (max_len - len(segments)), dtype=torch.long))
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # 预测位置不够就补零，mlm权重用来屏蔽填充损失
        all_mlm_weights.append(torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (max_num_mlm_preds - len(pred_positions)), dtype=torch.float32))
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)


### Step 4：自定义PyTorch数据集

In [19]:
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # 对每段落，按词分词
        paragraphs = [d2l.tokenize(paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs for sentence in paragraph]
        # 构建词表，过滤低频词
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=['<pad>', '<mask>', '<cls>', '<sep>'])
        # 生成NSP数据（每个样本：tokens, segments, is_next标签）
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(paragraph, paragraphs, self.vocab, max_len))
        # 生成MLM数据（每个样本：mask过的tokens、mask位置、mask原词……）
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab) + (segments, is_next)) for tokens, segments, is_next in examples]
        # 对所有样本做pad，得到统一长度
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx], self.valid_lens[idx],
                self.all_pred_positions[idx], self.all_mlm_weights[idx],
                self.all_mlm_labels[idx], self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)


### Step 5：数据加载函数

In [20]:
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集并转为DataLoader格式"""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(
        train_set, batch_size, shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

### Step 6：简单测试数据输出

In [21]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
     mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,
          nsp_y.shape)
    break

BadZipFile: File is not a zip file

In [11]:
len(vocab)

NameError: name 'vocab' is not defined